In [2]:
import pandas as pd
import numpy as np
import jieba
from collections import defaultdict
from sklearn.feature_extraction.text import CountVectorizer

## 将同一个id的多行的数据，变为一行一个id的数据

### 四分类

In [79]:
def get_deafult3():
    return defaultdict(lambda: 3)
table = defaultdict(get_deafult3)

In [80]:
with open('../data/train.csv', encoding='utf-8') as f:
    f.readline()
    for line in f.readlines():
        content_id, content, subject, sentiment_value, sentiment_word = line.split(',')
        table['content'][content_id] = content
        table[subject][content_id] = int(sentiment_value)+1
        word = 'word_{}'.format(subject)
        # table[word][content_id] = sentiment_word

In [81]:
df = pd.DataFrame.from_dict(table)

In [82]:
df.head(2)

,content,价格,内饰,动力,外观,安全性,操控,油耗,空间,舒适性,配置
01DqbWMwRjfBxLYz,自动启停对车没好处，等红灯挂N挡，轻刹车。,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN
01X4vSqHci6NPYBy,开的2.0？在西藏动力如何？,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [83]:
df.fillna(3, inplace=True)
df.rename_axis('content_id', inplace=True)

In [84]:
df.head(2)

,content,价格,内饰,动力,外观,安全性,操控,油耗,空间,舒适性,配置
content_id,,,,,,,,,,,
01DqbWMwRjfBxLYz,自动启停对车没好处，等红灯挂N挡，轻刹车。,3.0,3.0,3.0,3.0,3.0,1.0,3.0,3.0,3.0,3.0
01X4vSqHci6NPYBy,开的2.0？在西藏动力如何？,3.0,3.0,1.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0


In [85]:
df.to_csv('../data/input4.csv')

### 二分类

In [199]:
def get_deafult0():
    return defaultdict(lambda: 0)
table = defaultdict(get_deafult0)

In [200]:
with open('../data/train.csv', encoding='utf-8') as f:
    f.readline()
    for line in f.readlines():
        content_id, content, subject, sentiment_value, sentiment_word = line.split(',')
        table['content'][content_id] = content
        table[subject][content_id] = 1
        word = 'word_{}'.format(subject)
        # table[word][content_id] = sentiment_word
df = pd.DataFrame.from_dict(table)
df.head(2)

,content,价格,内饰,动力,外观,安全性,操控,油耗,空间,舒适性,配置
01DqbWMwRjfBxLYz,自动启停对车没好处，等红灯挂N挡，轻刹车。,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN
01X4vSqHci6NPYBy,开的2.0？在西藏动力如何？,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [201]:
df.fillna(0, inplace=True)
df.rename_axis('content_id', inplace=True)
df.head(2)

,content,价格,内饰,动力,外观,安全性,操控,油耗,空间,舒适性,配置
content_id,,,,,,,,,,,
01DqbWMwRjfBxLYz,自动启停对车没好处，等红灯挂N挡，轻刹车。,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
01X4vSqHci6NPYBy,开的2.0？在西藏动力如何？,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [202]:
df.to_csv('../data/input2.csv')

不平衡权重

In [205]:
for col in ['价格', '内饰', '动力', '外观', '安全性', '操控', '油耗', '空间', '舒适性', '配置']:
    pos = df[df[col]!=0].shape[0]
    neg = df.shape[0] - pos
    print("'{}': {},".format(col, neg/pos))

'价格': 5.512175962293794,
'内饰': 14.466417910447761,
'动力': 2.034407027818448,
'外观': 15.952965235173824,
'安全性': 13.467713787085515,
'操控': 7.001930501930502,
'油耗': 6.66173752310536,
'空间': 17.755656108597286,
'舒适性': 7.904403866809882,
'配置': 8.718640093786636,


In [ ]:

# df = pd.read_csv('../data/train.csv')

# df.groupby('subject').describe()

In [ ]:

# word_set = set()


# # In[7]:


# for line in df['content']:
#     words = jieba.cut(line)
#     word_set.update(words)


# # In[10]:


# in_count = 0
# with open('../data/merge_sgns_bigram_char300.txt', encoding='utf-8') as f, open('../data/cliped_vec.vec', 'w', encoding='utf-8') as outf:
#     f.readline()
#     while True:
#         line = f.readline()
#         if not line:
#             break
#         try:
#             word, vec = line.split(' ', 1)
#         except:
#             print(line)
#             continue
#         if word in word_set:
#             in_count += 1
#             outf.write(line)
#         else:
#             print(word)
# print('total have ', in_count, 'words in word embedding')


### 得到每个文档中tfidf比较重要的term，去重之后作为０/１特征

In [25]:
from sklearn.feature_extraction.text import TfidfTransformer

In [26]:
df = pd.read_csv('../data/input.csv')
df.head(5)

,content_id,content,价格,内饰,动力,外观,安全性,操控,油耗,空间,舒适性,配置
0,01DqbWMwRjfBxLYz,自动启停对车没好处，等红灯挂N挡，轻刹车。,3.0,3.0,3.0,3.0,3.0,1.0,3.0,3.0,3.0,3.0
1,01X4vSqHci6NPYBy,开的2.0？在西藏动力如何？,3.0,3.0,1.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
2,02VHu4amZkpAGTb7,常用配件价格不贵，跟大众差不多。而且常用配件都有备货，如果不太常用的可能需要等，但也绝不是从...,1.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
3,038JPMTIu1tRcZ2p,4万公里，悲催的11。。。市区每天办事，堵成汪。 哈哈哈。如果以前的锐志，起码13~14,3.0,3.0,3.0,3.0,3.0,3.0,1.0,3.0,3.0,3.0
4,03AUEJcWODxb9ZHa,建议最好使用原厂，subaru的发动机的冷却方式更别的车不一样，所以原厂更适合。,3.0,3.0,1.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0


In [27]:
def load_stopwords(filepath):
    with open(filepath, encoding='utf-8') as f:
        stops = set([v.strip() for v in f.readlines()])
    return stops
stops = load_stopwords('../data/stopwords.txt')
def seg_stop_merge(text):
    words = set(jieba.cut(text))
    words -= stops
    text = ' '.join(list(words))
    return text

In [28]:
test_df = pd.read_csv('../data/test_public.csv')
test_df.head(3)

,content_id,content
0,XuPwKCnA2fqNh5vm,欧蓝德，价格便宜，森林人太贵啦！
1,2jNbDn85goX3IuPE,楼主什么时候提的车，南昌优惠多少啊
2,hLgEADQ8sUnvGFK9,吉林，2.5优惠20000，送三年九次保养，贴膜


In [29]:
train_corpus = df['content'].apply(lambda text: seg_stop_merge(text))

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.868 seconds.
Prefix dict has been built succesfully.


In [30]:
test_corpus = test_df['content'].apply(lambda text: seg_stop_merge(text))

In [31]:
corpus = pd.concat([train_corpus, test_corpus])

In [32]:
vectorizer = CountVectorizer()

X = vectorizer.fit_transform(corpus)

word = vectorizer.get_feature_names()

In [35]:
# X.toarray()

In [36]:
transformer = TfidfTransformer()

tfidf = transformer.fit_transform(X)

TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)

In [38]:
# tfidf.toarray()

In [ ]:
term_weights = []
word = vectorizer.get_feature_names()#获取词袋模型中的所有词语
weight = tfidf.toarray()#将tf-idf矩阵抽取出来，元素a[i][j]表示j词在i类文本中的tf-idf权重
for i in range(len(weight)):#打印每类文本的tf-idf词语权重，第一个for遍历所有文本，第二个for便利某一类文本下的词语权重
    # print("-------这里输出第",i,u"类文本的词语tf-idf权重------")
    for j in range(len(word)):
        # print(word[j],weight[i][j])
        if weight[i][j] > 0:
            term_weights.append((word[j], weight[i][j]))


In [40]:
print(df.shape, test_df.shape, corpus.shape)

(8290, 12) (2364, 2) (10654,)


In [41]:
term_weights.sort(key=lambda v:v[1], reverse=True)

In [42]:
len(term_weights)

101234

In [43]:
import re
# alphanum = re.compile('[a-zA-Z0-9]')
alphanum = re.compile('[a-zA-Z]')

In [44]:
# important_words = set(v[0] for v, _ in zip(term_weights, range(3000)) if not alphanum.search(v[0]))
important_words = set(v[0] for v in term_weights if not alphanum.search(v[0]))

In [45]:
# 0 in important_words

In [46]:
len(important_words)

16324

In [47]:
with open('../data/tfidf1.txt', 'w', encoding='utf-8') as f:
    first = True
    for w in important_words:
        if first:
            first = False
        else:
            f.write('\n')
        f.write(w)
    

In [48]:
with open('../data/tfidf1.txt', encoding='utf-8') as f:
    important_words = [line.strip() for line in f.readlines()]

In [49]:
tdf = pd.DataFrame()
test_tdf = pd.DataFrame()
#tdf['content'] = df['content']
# tdf['content_id'] = df['content_id']
for w in important_words:
    tdf[w] = df['content'].apply(lambda text: 1 if text.find(w) != -1 else 0)
    test_tdf[w] = test_df['content'].apply(lambda text: 1 if text.find(w) != -1 else 0)

In [50]:
tdf.head(1)

,寄过来,黄色,空闲,四趟,车多,雪白,再就是,干什么,油到,叽响,...,车坏,该刹,后防,塞得,觉悟,气枪,购得,车真,中度,车龄长
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [54]:
invalid_cols = []
for col in tdf.columns:
    if tdf[tdf[col]>0].shape[0] < 5:
        invalid_cols.append(col)
print(len(invalid_cols))

12893


In [55]:
tdf.drop(columns=invalid_cols, inplace=True)
tdf.shape

(8290, 3431)

In [56]:
test_tdf.drop(columns=invalid_cols, inplace=True)
test_tdf.shape

(2364, 3431)

In [57]:
tdf.to_csv('../data/tfidf_train.csv', index=False)
test_tdf.to_csv('../data/tfidf_test.csv', index=False)

### 2gram 词 做tfidf

In [3]:
from sklearn.feature_extraction.text import TfidfTransformer

In [4]:
df = pd.read_csv('../data/input2.csv')
df.head(5)

,content_id,content,价格,内饰,动力,外观,安全性,操控,油耗,空间,舒适性,配置
0,01DqbWMwRjfBxLYz,自动启停对车没好处，等红灯挂N挡，轻刹车。,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,01X4vSqHci6NPYBy,开的2.0？在西藏动力如何？,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,02VHu4amZkpAGTb7,常用配件价格不贵，跟大众差不多。而且常用配件都有备货，如果不太常用的可能需要等，但也绝不是从...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,038JPMTIu1tRcZ2p,4万公里，悲催的11。。。市区每天办事，堵成汪。 哈哈哈。如果以前的锐志，起码13~14,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,03AUEJcWODxb9ZHa,建议最好使用原厂，subaru的发动机的冷却方式更别的车不一样，所以原厂更适合。,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
def load_stopwords(filepath):
    with open(filepath, encoding='utf-8') as f:
        stops = set([v.strip() for v in f.readlines()])
    return stops
stops = load_stopwords('../data/stopwords.txt')
def seg_stop_merge2(text):
    segs = jieba.lcut(text)
    words = set(segs)
    words -= stops
    words = list(words)
    words.sort(key=segs.index)
    words_2gram = ['{}{}'.format(w1, w2) for w1, w2 in zip(words[:-1], words[1:])]
    text = ' '.join(words_2gram)
    
    return text

In [6]:
test_df = pd.read_csv('../data/test_public.csv')
test_df.head(3)

,content_id,content
0,XuPwKCnA2fqNh5vm,欧蓝德，价格便宜，森林人太贵啦！
1,2jNbDn85goX3IuPE,楼主什么时候提的车，南昌优惠多少啊
2,hLgEADQ8sUnvGFK9,吉林，2.5优惠20000，送三年九次保养，贴膜


In [14]:
train_corpus = df['content'].apply(lambda text: seg_stop_merge2(text))
test_corpus = test_df['content'].apply(lambda text: seg_stop_merge2(text))
corpus = pd.concat([train_corpus, test_corpus])

In [15]:
vectorizer = CountVectorizer()

X = vectorizer.fit_transform(corpus)

word = vectorizer.get_feature_names()

In [16]:
transformer = TfidfTransformer()

tfidf = transformer.fit_transform(X)

In [19]:
term_weights = []
word = vectorizer.get_feature_names()#获取词袋模型中的所有词语
weight = tfidf.toarray()#将tf-idf矩阵抽取出来，元素a[i][j]表示j词在i类文本中的tf-idf权重
for i in range(len(weight)):#打印每类文本的tf-idf词语权重，第一个for遍历所有文本，第二个for便利某一类文本下的词语权重
    # print("-------这里输出第",i,u"类文本的词语tf-idf权重------")
    for j in range(len(word)):
        # print(word[j],weight[i][j])
        if weight[i][j] > 0:
            term_weights.append((word[j], weight[i][j]))


MemoryError: 

In [ ]:
len(term_weights)

In [ ]:
import re
# alphanum = re.compile('[a-zA-Z0-9]')
alphanum = re.compile('[a-zA-Z]')

In [ ]:
# important_words = set(v[0] for v, _ in zip(term_weights, range(3000)) if not alphanum.search(v[0]))
important_words = set(v[0] for v in term_weights if not alphanum.search(v[0]))
len(important_words)

In [ ]:
with open('../data/tfidf2.txt', 'w', encoding='utf-8') as f:
    first = True
    for w in important_words:
        if first:
            first = False
        else:
            f.write('\n')
        f.write(w)
    

In [ ]:
with open('../data/tfidf1.txt', encoding='utf-8') as f:
    important_words = [line.strip() for line in f.readlines()]

In [ ]:
tdf = pd.DataFrame()
test_tdf = pd.DataFrame()
#tdf['content'] = df['content']
# tdf['content_id'] = df['content_id']
for w in important_words:
    tdf[w] = df['content'].apply(lambda text: 1 if text.find(w) != -1 else 0)
    test_tdf[w] = test_df['content'].apply(lambda text: 1 if text.find(w) != -1 else 0)

In [ ]:
invalid_cols = []
for col in tdf.columns:
    if tdf[tdf[col]>0].shape[0] < 5:
        invalid_cols.append(col)
print(len(invalid_cols))

In [ ]:
tdf.drop(columns=invalid_cols, inplace=True)
tdf.shape

In [ ]:
test_tdf.drop(columns=invalid_cols, inplace=True)
test_tdf.shape

In [ ]:
tdf.to_csv('../data/tfidf2_train.csv', index=False)
test_tdf.to_csv('../data/tfidf2_test.csv', index=False)

In [ ]:


# for index in range(tdf.shape[0]):
#     for j in range(len(word)):
#         if weight[index][j] > 0 and (word[j] in important_words):
#             #print(word[j])
#             tdf.loc[index, word[j]] = weight[index][j]

            

In [ ]:
# tdf.head(5)

In [ ]:
# tdf['空档'].value_counts()[0]

In [ ]:
# meaningless_cols = []
# for col in tdf.columns:
#     if tdf[col].value_counts()[0] > tdf.shape[0]-5:
#         meaningless_cols.append(col)
# print(meaningless_cols)

In [ ]:
# print(len(meaningless_cols))

In [ ]:
# meaningful_cols = list(important_words-set(meaningless_cols))

In [ ]:
# train_tdf = tdf[meaningful_cols]

In [ ]:
# train_tdf.head(3)

In [ ]:
# train_tdf.to_csv('../data/tfidf_train.csv', index=False)
# meaningful_words = set(meaningful_cols)

In [ ]:
# test_tdf = pd.DataFrame()
# for w in meaningful_cols:
#     test_tdf[w] = np.array([0] * test_df.shape[0])

In [ ]:
# for i in range(test_tdf.shape[0]):
#     index = i + train_tdf.shape[0]
#     for j in range(len(word)):
#         if weight[index][j] > 0 and (word[j] in meaningful_words):
#             #print(word[j])
#             test_tdf.loc[i, word[j]] = weight[index][j]

       

In [ ]:
# test_tdf.head(4)

In [ ]:
# test_tdf.to_csv('../data/tfidf_test.csv', index=False)

### 利用规则提取的特征

In [31]:
import rule_utils as ru

In [32]:
subjects = {
    'price': '价格', 
    'interior': '内饰', 
    'power': '动力', 
    'surface': '外观', 
    'safety': '安全性', 
    'operation':'操控', 
    'gas' : '油耗', 
    'space': '空间', 
    'comfort': '舒适性', 
    'config': '配置'
}


In [33]:
matchers = [(sub, ru.match_subject('../rules/{}.txt'.format(sub))) for sub in subjects]

In [34]:
df = pd.read_csv('../data/input.csv')
df.head(5)

,content_id,content,价格,内饰,动力,外观,安全性,操控,油耗,空间,舒适性,配置
0,01DqbWMwRjfBxLYz,自动启停对车没好处，等红灯挂N挡，轻刹车。,3.0,3.0,3.0,3.0,3.0,1.0,3.0,3.0,3.0,3.0
1,01X4vSqHci6NPYBy,开的2.0？在西藏动力如何？,3.0,3.0,1.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
2,02VHu4amZkpAGTb7,常用配件价格不贵，跟大众差不多。而且常用配件都有备货，如果不太常用的可能需要等，但也绝不是从...,1.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
3,038JPMTIu1tRcZ2p,4万公里，悲催的11。。。市区每天办事，堵成汪。 哈哈哈。如果以前的锐志，起码13~14,3.0,3.0,3.0,3.0,3.0,3.0,1.0,3.0,3.0,3.0
4,03AUEJcWODxb9ZHa,建议最好使用原厂，subaru的发动机的冷却方式更别的车不一样，所以原厂更适合。,3.0,3.0,1.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0


In [40]:
df['content'].shape

(8290,)

In [35]:
test_df = pd.read_csv('../data/test_public.csv')
test_df.head(5)

,content_id,content
0,XuPwKCnA2fqNh5vm,欧蓝德，价格便宜，森林人太贵啦！
1,2jNbDn85goX3IuPE,楼主什么时候提的车，南昌优惠多少啊
2,hLgEADQ8sUnvGFK9,吉林，2.5优惠20000，送三年九次保养，贴膜
3,nZmM7LQsfr03wUaz,便宜2万的豪华特装，实用配制提升，优惠还给力，确实划算。
4,pwd8MnrthDqLZafe,如果实在想买就等车展期间，优惠2万，我24.98万入的2.5豪


In [41]:
rdf = pd.DataFrame()
for sub in subjects:
    rdf[sub] = np.array([0] * df.shape[0])

In [42]:
for i, value in enumerate(df['content']):
    has = False
    for sub, matcher in matchers:          
        if matcher(value):
            rdf.loc[i, sub] = 1              


In [43]:
rdf.shape

(8290, 10)

In [44]:
# rdf.fillna(0, inplace=True)
rdf.head(5)

,price,interior,power,surface,safety,operation,gas,space,comfort,config
0,0,0,0,0,1,0,0,0,0,0
1,0,0,1,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0
4,0,0,1,0,0,0,0,0,0,0


In [45]:
rdf.to_csv('../data/rule_train.csv', index=False) 

In [46]:
test_rdf = pd.DataFrame()
for sub in subjects:
    test_rdf[sub] = np.array([0] * test_df.shape[0])

In [47]:
for i, value in enumerate(test_df['content']):
    has = False
    for sub, matcher in matchers:          
        if matcher(value):
            test_rdf.loc[i, sub] = 1              


In [48]:
# test_rdf.fillna(0, inplace=True)
test_rdf.head(3)

,price,interior,power,surface,safety,operation,gas,space,comfort,config
0,1,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0


In [49]:
test_rdf.to_csv('../data/rule_test.csv', index=False) 

### 合并规则和ｔｆｉｄｆ                                                                                                                                                                                     

In [58]:
tdf = pd.read_csv('../data/tfidf_train.csv')
rdf = pd.read_csv('../data/rule_train.csv')
trdf = pd.merge(tdf, rdf, left_index=True, right_index=True)
trdf.head(3)

,车多,再就是,干什么,没得说,降档,少有,主力,满满的,算是,第一个,...,config_9,config_10,config_11,config_12,config_13,config_14,config_15,config_16,config_17,config_18
0,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [59]:
test_tdf = pd.read_csv('../data/tfidf_test.csv')
test_rdf = pd.read_csv('../data/rule_test.csv')
test_trdf = pd.merge(test_tdf, test_rdf, left_index=True, right_index=True)
test_trdf.head(3)

,车多,再就是,干什么,没得说,降档,少有,主力,满满的,算是,第一个,...,config_9,config_10,config_11,config_12,config_13,config_14,config_15,config_16,config_17,config_18
0,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [60]:
trdf.to_csv('../data/merge_train.csv', index=False)

In [61]:
test_trdf.to_csv('../data/merge_test.csv', index=False)

In [62]:
rdf.shape

(8290, 110)

In [63]:
trdf.shape

(8290, 3541)

In [64]:
tdf.shape

(8290, 3431)

### 使用doc2vec作为特征(垃圾，下面的别看了)

In [94]:
from gensim.test.utils import common_texts
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from gensim.test.utils import get_tmpfile

In [176]:
feature_size = 300

In [177]:
df = pd.read_csv('../data/input.csv')
df.head(5)

,content_id,content,价格,内饰,动力,外观,安全性,操控,油耗,空间,舒适性,配置
0,01DqbWMwRjfBxLYz,自动启停对车没好处，等红灯挂N挡，轻刹车。,3.0,3.0,3.0,3.0,3.0,1.0,3.0,3.0,3.0,3.0
1,01X4vSqHci6NPYBy,开的2.0？在西藏动力如何？,3.0,3.0,1.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
2,02VHu4amZkpAGTb7,常用配件价格不贵，跟大众差不多。而且常用配件都有备货，如果不太常用的可能需要等，但也绝不是从...,1.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
3,038JPMTIu1tRcZ2p,4万公里，悲催的11。。。市区每天办事，堵成汪。 哈哈哈。如果以前的锐志，起码13~14,3.0,3.0,3.0,3.0,3.0,3.0,1.0,3.0,3.0,3.0
4,03AUEJcWODxb9ZHa,建议最好使用原厂，subaru的发动机的冷却方式更别的车不一样，所以原厂更适合。,3.0,3.0,1.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0


In [178]:
sign_ptn = re.compile(r'[\s+\.\!\/_,$%^*(+\"\')]+|[+——()?【】“”！，。×》〕:《〈？、~@#￥%……&*（）]+')
def pre_doc(text):
    text = sign_ptn.sub(' ', text)
    return jieba.lcut(text)

In [179]:
documents = [TaggedDocument(pre_doc(doc), [i]) for i, doc in enumerate(df['content'])]

In [180]:
model = Doc2Vec(documents, vector_size=feature_size, window=5, min_count=5, workers=4)

In [181]:

fname = get_tmpfile("my_doc2vec_model")
model.save(fname)

In [182]:
model = Doc2Vec.load(fname)  # you can continue training with the loaded model!

In [183]:
model.train(documents, total_examples=model.corpus_count, epochs=3)

In [184]:
test_df = pd.read_csv('../data/test_public.csv')

In [185]:
ddf = pd.DataFrame(columns=['dim{}'.format(i) for i in range(feature_size)])

In [186]:
for i, line in enumerate(df['content']):
    vector = model.infer_vector(pre_doc(line))
    ddf.loc[i] = vector

In [187]:
ddf.head(3)

,dim0,dim1,dim2,dim3,dim4,dim5,dim6,dim7,dim8,dim9,...,dim290,dim291,dim292,dim293,dim294,dim295,dim296,dim297,dim298,dim299
0,0.005715,0.014842,-0.029451,0.059574,-0.024024,-0.032527,-0.023325,0.007485,-0.019683,0.061700,...,-0.002086,0.016926,-0.056125,-0.011747,-0.030771,0.025235,-0.013046,-0.011786,0.012418,-0.002628
1,-0.015752,0.023767,-0.013724,0.038414,-0.017228,-0.040916,0.015745,0.012659,-0.008252,0.042249,...,0.001086,0.007130,-0.046009,-0.005499,-0.027451,0.006718,-0.000467,-0.000592,-0.012604,0.008060
2,0.008113,0.018890,-0.007853,0.019682,-0.017544,-0.035542,-0.029634,0.023031,0.020157,0.012318,...,-0.022084,-0.023343,-0.029284,-0.000827,0.004234,0.015758,-0.059801,-0.023415,-0.024622,0.010148


In [188]:
ddf.to_csv('../data/doc2vec_train.csv', index=False)

In [189]:
test_ddf = pd.DataFrame(columns=['dim{}'.format(i) for i in range(feature_size)])
for i, line in enumerate(test_df['content']):
    vector = model.infer_vector(pre_doc(line))
    test_ddf.loc[i] = vector
test_ddf.head(3)

,dim0,dim1,dim2,dim3,dim4,dim5,dim6,dim7,dim8,dim9,...,dim290,dim291,dim292,dim293,dim294,dim295,dim296,dim297,dim298,dim299
0,0.001245,0.005962,0.004733,0.008307,-0.019745,-0.003737,0.006057,-0.018863,-0.030885,-0.002766,...,-0.007964,0.004064,-0.012269,-0.022594,-0.004643,0.005034,-0.001102,-0.011358,0.006864,-0.023441
1,-0.002187,0.007707,-0.001188,-0.022830,0.002774,0.014018,-0.001150,0.005699,0.016225,-0.035436,...,0.002870,-0.032379,0.006623,0.002371,0.001698,-0.001728,0.004181,-0.021773,0.000885,-0.005230
2,-0.016399,0.008509,-0.000153,-0.005113,0.005511,-0.001086,0.010200,0.007638,0.016311,0.014070,...,0.022500,0.002564,-0.001131,0.006677,0.000816,0.002319,0.037299,0.022175,0.012347,0.005067


In [190]:
test_ddf.to_csv('../data/doc2vec_test.csv', index=False)

### 合并doc2vec和之前的merge特征

In [191]:
ddf = pd.read_csv('../data/doc2vec_train.csv')
mdf = pd.read_csv('../data/merge_train.csv')
mddf = pd.merge(mdf, ddf, left_index=True, right_index=True)
mddf.head(3)

,车多,再就是,干什么,没得说,降档,少有,主力,满满的,算是,第一个,...,dim290,dim291,dim292,dim293,dim294,dim295,dim296,dim297,dim298,dim299
0,0,0,0,0,0,0,0,0,0,0,...,-0.002086,0.016926,-0.056125,-0.011747,-0.030771,0.025235,-0.013046,-0.011786,0.012418,-0.002628
1,0,0,0,0,0,0,0,0,0,0,...,0.001086,0.007130,-0.046009,-0.005499,-0.027451,0.006718,-0.000467,-0.000592,-0.012604,0.008060
2,0,0,0,0,0,0,0,0,0,0,...,-0.022084,-0.023343,-0.029284,-0.000827,0.004234,0.015758,-0.059801,-0.023415,-0.024622,0.010148


In [196]:
mddf.to_csv('../data/merge1_train.csv', index=False)

In [194]:
test_ddf = pd.read_csv('../data/doc2vec_test.csv')
test_mdf = pd.read_csv('../data/merge_test.csv')
test_mddf = pd.merge(test_mdf, test_ddf, left_index=True, right_index=True)
test_mddf.head(3)

,车多,再就是,干什么,没得说,降档,少有,主力,满满的,算是,第一个,...,dim290,dim291,dim292,dim293,dim294,dim295,dim296,dim297,dim298,dim299
0,0,0,0,0,0,0,0,0,0,0,...,-0.007964,0.004064,-0.012269,-0.022594,-0.004643,0.005034,-0.001102,-0.011358,0.006864,-0.023441
1,0,0,0,0,0,0,0,0,0,0,...,0.002870,-0.032379,0.006623,0.002371,0.001698,-0.001728,0.004181,-0.021773,0.000885,-0.005230
2,0,0,0,0,0,0,0,0,0,0,...,0.022500,0.002564,-0.001131,0.006677,0.000816,0.002319,0.037299,0.022175,0.012347,0.005067


In [197]:
test_mddf.to_csv('../data/merge1_test.csv', index=False)